In [1]:
import numpy as np 
import pandas as pd 
import os


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,BatchNormalization,Dropout
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix 
import seaborn as sns
import cv2
import warnings
warnings.filterwarnings('ignore')

In [3]:
def load_and_preprocess_images(folder_path, subfolders, image_size=(224, 224)):
    
    label_map = {subfolder: idx for idx, subfolder in enumerate(subfolders)}
    
    images = []
    labels = []
    
    for subfolder in subfolders:
        subfolder_path = os.path.join(folder_path, subfolder)
        for filename in os.listdir(subfolder_path):
            img_path = os.path.join(subfolder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                
                img_resized = cv2.resize(img, image_size)
                images.append(img_resized)
                labels.append(label_map[subfolder])
    
    images = np.array(images)
    labels = np.array(labels)
    
    images = images.astype('float32') / 255.0
    
    return images, labels

#/kaggle/input/fruits-classification/Fruits Classification/train/Apple

#/kaggle/input/fruits-classification/Fruits Classification/train/Apple/Apple (1).jpeg

In [4]:
train_dir = "/kaggle/input/fruits-classification/Fruits Classification/train"
test_dir = "/kaggle/input/fruits-classification/Fruits Classification/test"
valid_dir = "/kaggle/input/fruits-classification/Fruits Classification/valid"

subfolders = ['Apple', 'Banana', 'Grape', 'Mango','Strawberry']

x_train, y_train = load_and_preprocess_images(train_dir, subfolders)
x_test , y_test = load_and_preprocess_images(test_dir, subfolders)
x_valid, y_valid = load_and_preprocess_images(valid_dir, subfolders)

In [5]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)
print(x_valid.shape,y_valid.shape)

(9700, 224, 224, 3) (9700,)
(100, 224, 224, 3) (100,)
(200, 224, 224, 3) (200,)


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 9700 images belonging to 5 classes.
Found 100 images belonging to 5 classes.


In [7]:
#CNN building:
num_classes = len(train_generator.class_indices)

model = Sequential([
    Conv2D(128, (3, 3), activation='relu', padding='same', input_shape=(256, 256, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),


    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),


    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),


    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),


    Flatten(),

    Dense(512, activation='relu'),
    BatchNormalization(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(5, activation='softmax')
])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 128)  │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256, 256, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 16, 16, 32)     │        36,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 16, 16, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 1,422,181 (5.43 MB)

 Trainable params: 1,420,069 (5.42 MB)

 Non-trainable params: 2,112 (8.25 KB)

In [8]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['Accuracy'])

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('model_checkpoint.weights.h5', save_best_only=False, save_weights_only=True, verbose=1)



In [11]:
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=test_generator,
    batch_size=32,
    callbacks=[checkpoint]
)

Epoch 1/100


I0000 00:00:1727096427.806329     117 service.cc:145] XLA service 0x7bcfe0007eb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727096427.806398     117 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  2/304 ━━━━━━━━━━━━━━━━━━━━ 29s 97ms/step - Accuracy: 0.3359 - loss: 2.0542   

I0000 00:00:1727096439.669364     117 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 450ms/step - Accuracy: 0.4233 - loss: 1.5174
Epoch 1: saving model to model_checkpoint.weights.h5
304/304 ━━━━━━━━━━━━━━━━━━━━ 158s 455ms/step - Accuracy: 0.4235 - loss: 1.5167 - val_Accuracy: 0.2300 - val_loss: 2.0416
Epoch 2/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step - Accuracy: 0.5646 - loss: 1.0893
Epoch 2: saving model to model_checkpoint.weights.h5
304/304 ━━━━━━━━━━━━━━━━━━━━ 136s 435ms/step - Accuracy: 0.5646 - loss: 1.0893 - val_Accuracy: 0.5400 - val_loss: 1.1286
Epoch 3/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step - Accuracy: 0.5962 - loss: 1.0189
Epoch 3: saving model to model_checkpoint.weights.h5
304/304 ━━━━━━━━━━━━━━━━━━━━ 136s 436ms/step - Accuracy: 0.5962 - loss: 1.0188 - val_Accuracy: 0.6400 - val_loss: 0.9210
Epoch 4/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step - Accuracy: 0.6328 - loss: 0.9366
Epoch 4: saving model to model_checkpoint.weights.h5
304/304 ━━━━━━━━━━━━━━━━━━━━ 136s 437ms/step - Accuracy: 0.6328 - loss: 0.9366 - 

In [14]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc *100} %')


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - Accuracy: 0.9189 - loss: 0.3251
Test accuracy: 93.00000071525574 %


In [15]:
model.save('fruit_recognizer.h5')